In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# import os
# #Change the current working directory to the path of Google Cloud Drive
# path="/content/drive/My Drive/Colab Notebooks/"
# os.chdir(path)
# os.listdir(path)
# #Use the wget command to download the dataset to this path
# !wget https://fordatis.fraunhofer.de/bitstream/fordatis/151.2/1/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy as sc
import zipfile
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
#url='https://fordatis.fraunhofer.de/bitstream/fordatis/151.2/1/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip'

In [ ]:
# !unzip gdrive/My\ Drive/Colab\ Notebooks/data/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip

In [ ]:
url='/content/drive/MyDrive/Colab Notebooks/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip'
use_reference_models = True
model_path = '/content/drive/MyDrive/References_cnn'

In [ ]:
with zipfile.ZipFile(url, 'r') as f:
    with f.open('0D.csv', 'r') as c:
        data0D = pd.read_csv(c)
    with f.open('0E.csv', 'r') as c:
        data0E = pd.read_csv(c)
    with f.open('1D.csv', 'r') as c:
        data1D = pd.read_csv(c)
    with f.open('1E.csv', 'r') as c:
        data1E = pd.read_csv(c)
    with f.open('2D.csv', 'r') as c:
        data2D = pd.read_csv(c)
    with f.open('2E.csv', 'r') as c:
        data2E = pd.read_csv(c)
    with f.open('3D.csv', 'r') as c:
        data3D = pd.read_csv(c)
    with f.open('3E.csv', 'r') as c:
        data3E = pd.read_csv(c)
    with f.open('4D.csv', 'r') as c:
        data4D = pd.read_csv(c)
    with f.open('4E.csv', 'r') as c:
        data4E = pd.read_csv(c)

In [ ]:
skip = 50000
data0D = data0D.iloc[skip:,:]
data1D = data1D.iloc[skip:,:]
data2D = data2D.iloc[skip:,:]
data3D = data3D.iloc[skip:,:]
data4D = data4D.iloc[skip:,:]

data0E = data0E.iloc[skip:,:]
data1E = data1E.iloc[skip:,:]
data2E = data2E.iloc[skip:,:]
data3E = data3E.iloc[skip:,:]
data4E = data4E.iloc[skip:,:]

In [ ]:
labels = {'no_unbalance':0, 'unbalance':1}
sensor = 'Vibration_1'
samples_per_second = 4096
seconds_per_analysis = 1.0
window = int(samples_per_second*seconds_per_analysis)

def get_features(data, label):
    n = int(np.floor(len(data)/window))
    data = data[:int(n)*window]
    X = data.values.reshape((n, window))
    y = np.ones(n)*labels[label]
    return X,y

X0,y0 = get_features(data0D[sensor], "no_unbalance")
X1,y1 = get_features(data1D[sensor], "unbalance")
X2,y2 = get_features(data2D[sensor], "unbalance")
X3,y3 = get_features(data3D[sensor], "unbalance")
X4,y4 = get_features(data4D[sensor], "unbalance")
X=np.concatenate([X0, X1, X2, X3, X4])
y=np.concatenate([y0, y1, y2, y3, y4])

X0_val, y0_val = get_features(data0E[sensor], "no_unbalance")
X1_val, y1_val = get_features(data1E[sensor], "unbalance")
X2_val, y2_val = get_features(data2E[sensor], "unbalance")
X3_val, y3_val = get_features(data3E[sensor], "unbalance")
X4_val, y4_val = get_features(data4E[sensor], "unbalance")
X_val=np.concatenate([X0_val, X1_val, X2_val, X3_val, X4_val])
y_val=np.concatenate([y0_val, y1_val, y2_val, y3_val, y4_val])

In [ ]:
print(X0)

[[ 2.1606684e-02  2.2007227e-02  2.1793842e-02 ...  1.6417503e-02
   1.6212463e-02  1.5910864e-02]
 [ 1.6222000e-02  1.5923977e-02  1.6121864e-02 ...  1.2435913e-02
   1.2710094e-02  1.2711287e-02]
 [ 1.2282133e-02  1.2310743e-02  1.2122393e-02 ...  9.3388557e-03
   9.8717213e-03  9.6476078e-03]
 ...
 [ 1.1742115e-03  1.1193752e-03  1.2993813e-03 ...  8.2612038e-04
   4.1604042e-04  8.5115433e-04]
 [ 6.6399574e-04  4.4107437e-05 -1.9907951e-04 ...  3.7312508e-04
   9.3460083e-04  1.1575222e-03]
 [ 8.2135201e-04  9.4175339e-04  6.2346458e-04 ...  1.1920929e-03
   6.9975853e-04  5.7458878e-04]]


In [ ]:
print(X.shape, y.shape, X_val.shape, y_val.shape)

(32166, 4096) (32166,) (8359, 4096) (8359,)


In [ ]:
from sklearn.model_selection import train_test_split
train_test_ratio = 0.9
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 1-train_test_ratio, random_state = 0)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(28949, 4096, 1) (28949,) (3217, 4096, 1) (3217,)


In [ ]:
from tensorflow import keras
from keras.layers import BatchNormalization,LeakyReLU,Dense,Dropout
from keras.layers import Input,Conv1D,MaxPooling1D,Flatten,ReLU
from keras.models import Sequential, load_model, Model

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2

weight_for_0 = len(y)/(2*len(y[y==0]))
weight_for_1 = len(y)/(2*len(y[y==1]))
class_weight = {0: weight_for_0, 1: weight_for_1}

def train_models(n_conv_layers):
    #n_conv_layers = 3 # [1,2,3,4]
    n_dense_units = 128
    dropout_rate = 0.0
    use_batch_normalization = True # [True, False]
    filter_size = 9 # [5,7,9]
    learning_rate = 0.0001
    n_epochs = 100 # [50,100,200]

    X_in = Input(shape=(X_train.shape[1],1))
    x = X_in
    for j in range(n_conv_layers):
        print(j)
        x = Conv1D(filters=(j+1)*10,
                   kernel_size=filter_size,
                   strides=1,
                   activation='linear',
                   kernel_initializer='he_uniform')(x)
        if use_batch_normalization:
            x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.05)(x)
        x = MaxPooling1D(pool_size=5, strides=2)(x)
    x = Flatten()(x)
    x = Dense(units = n_dense_units, activation='linear')(x)
    x = ReLU()(x)
    x = Dropout(rate=dropout_rate)(x)
    X_out = Dense(units = 1, activation = 'sigmoid')(x)
    classifier = Model(X_in, X_out)
    classifier.summary()

    best_model_filepath = f"{model_path}/cnn_{n_conv_layers}_layers.h5"
    checkpoint = ModelCheckpoint(best_model_filepath, monitor='val_loss',
                                 verbose=1, save_best_only=True, mode='min')

    classifier.compile(optimizer = Adam(lr=learning_rate), loss = 'binary_crossentropy',
                       metrics = ['accuracy'])

    classifier.fit(X_train, y_train, epochs = n_epochs, batch_size = 64,
                   validation_data=(X_test, y_test), callbacks=[checkpoint],
                   class_weight=class_weight)
    classifier = load_model(best_model_filepath)
    score = classifier.evaluate(X_val, y_val)

In [ ]:
if not True:
    for i in range(1,5):
        train_models(i)

In [ ]:
X_val_1 = X_val[:len(y0_val),:,:]
X_val_2 = X_val[len(y0_val):len(y0_val)+len(y1_val),:,:]
X_val_3 = X_val[len(y0_val)+len(y1_val):len(y0_val)+len(y1_val)+
                len(y2_val),:,:]
X_val_4 = X_val[len(y0_val)+len(y1_val)+len(y2_val):len(y0_val)+
                len(y1_val)+len(y2_val)+len(y3_val),:,:]
X_val_5 = X_val[len(y0_val)+len(y1_val)+len(y2_val)+len(y3_val):len(y0_val)+
                len(y1_val)+len(y2_val)+len(y3_val)+len(y4_val),:,:]

In [ ]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
accuracies = []
accuracies_all = []
for layer_n in range(1,5):

    filepath = f"{model_path}/cnn_{layer_n}_layers.h5"
    model_i = load_model(filepath)

    val_acc_1 = model_i.evaluate(X_val_1, y0_val)[1]
    val_acc_2 = model_i.evaluate(X_val_2, y1_val)[1]
    val_acc_3 = model_i.evaluate(X_val_3, y2_val)[1]
    val_acc_4 = model_i.evaluate(X_val_4, y3_val)[1]
    val_acc_5 = model_i.evaluate(X_val_5, y4_val)[1]
    val_acc_all = model_i.evaluate(X_val, y_val)[1]
    accuracies_layer_i = [val_acc_1, val_acc_2, val_acc_3, val_acc_4, val_acc_5]
    accuracies.append(accuracies_layer_i)
    accuracies_all.append(val_acc_all)

accuracies = np.array(accuracies)
accuracies_all = np.array(accuracies_all)

OSError: ignored

In [ ]:
accuracies_all

In [ ]:
fig=plt.figure(figsize=(12,8))
ax1=plt.subplot(111, title = "Unbalance Classification Trained with all Unbalances")
unbalances = np.array([0, 4.59e-5, 6.07e-5,7.55e-5,1.521e-4])
ax1.plot(1e6*unbalances, accuracies[0,:], label=f"1 layer, accuracy: {100.0*accuracies_all[0]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, accuracies[1,:], label=f"2 layers, accuracy: {100.0*accuracies_all[1]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, accuracies[2,:], label=f"3 layers, accuracy: {100.0*accuracies_all[2]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, accuracies[3,:], label=f"4 layers, accuracy: {100.0*accuracies_all[3]:.1f}%", marker="+", ls="--")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Unbalance Factor [mm g]")
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim([0.45, 1.05])
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model(f"{model_path}/cnn_1_layers.h5")
model2 = load_model(f"{model_path}/cnn_2_layers.h5")
model3 = load_model(f"{model_path}/cnn_3_layers.h5")
model4 = load_model(f"{model_path}/cnn_4_layers.h5")

In [ ]:
def v2rpm(v):
    return 212*v + 209

from scipy.stats import mode
# 3s ramp up
fade_in = np.arange(0.0, 4.0, 4.0/(3*4096))
# complete voltage sweep
measurement_cycle = np.repeat(np.arange(4.0, 8.2, 0.1), 4096*20.0)
# measurement: start-up + 2 voltage sweeps
measurement = np.concatenate([fade_in, np.tile(measurement_cycle,3)])
# select the data as actually used
measurement1 = measurement[50000:]
measurement1 = measurement1[:int(len(measurement1)/4096)*4096].reshape(-1,4096)
voltages_measurement = mode(measurement1, axis=1)[0]
voltages_used = np.concatenate([voltages_measurement[:len(X_val_1)],
                                voltages_measurement[:len(X_val_2)],
                                voltages_measurement[:len(X_val_3)],
                                voltages_measurement[:len(X_val_4)],
                                voltages_measurement[:len(X_val_5)]])
rpms_used = v2rpm(voltages_used)

In [ ]:
rpm_borders = np.arange(1050, 1975, 25)
errors_per_rpm_range1 = []
errors_per_rpm_range2 = []
errors_per_rpm_range3 = []
errors_per_rpm_range4 = []
for i in range(len(rpm_borders)-1):
    eval_inds = np.where((rpms_used>rpm_borders[i])&(rpms_used<rpm_borders[i+1]))[0]
    errors_per_rpm_range1.append(
        1-np.mean(np.abs(np.int32(model1.predict(X_val[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range2.append(
        1-np.mean(np.abs(np.int32(model2.predict(X_val[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range3.append(
        1-np.mean(np.abs(np.int32(model3.predict(X_val[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range4.append(
        1-np.mean(np.abs(np.int32(model4.predict(X_val[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))

In [ ]:
fig=plt.figure(figsize=(12,8))
ax1=plt.subplot(111, title = "Rotation Speed Dependent Evaluation")
ax1.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range1, marker="+", ls="--", label="1 conv. layer")
ax1.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range2, marker="+", ls="--", label="2 conv. layers")
ax1.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range3, marker="+", ls="--", label="3 conv. layers")
ax1.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range4, marker="+", ls="--", label="4 conv. layers")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Rotation Speed [rpm]")
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim([0.45, 1.05])
plt.grid(True)
plt.show()